In [1]:
# State

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)
from quara.data_analysis.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.data_analysis.weighted_relative_entropy import (
    WeightedRelativeEntropy,
    WeightedRelativeEntropyOption,
)
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_povm,
    get_y_povm,
    get_z_povm,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting
from quara.data_analysis.generation_setting import QOperationGenerationSetting, DepolarizedQOperationGenerationSetting
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

from quara.settings import Settings
Settings.get_atol()

1e-13

In [2]:
%reload_ext autoreload
%autoreload 2

- test_settingのリストを作る
- 1つのtest_settingから、QOperationGenerationSettingのリストを作る
- QOperationGenerationSettingから、qoperationのサンプルをn_sample個作る
- qoperationのサンプルから、StandardQTomographySimulationSettingを作る
- 推定を実行する
- SimulationCheckで結果をチェックする
- 結果を格納する or 保存する 

In [3]:
from collections import namedtuple

In [4]:
# test_settingのリストを作る
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Base of Tester Object
povm_x = get_x_povm(c_sys)
povm_y = get_y_povm(c_sys)
povm_z = get_z_povm(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

# Base of True Object
base_state =  get_z0_1q(c_sys)

In [5]:
# シミュレーションに必要な設定を作る
# TODO: StandardQTomographySimulationSettingクラスを使うかどうかの検討
case_names = [
    "Linear (True)",
    "Linear (False)",
    "ProjectedLinear (True)",
    "ProjectedLinear (False)",
    "Maximum-Likelihood (True)",
    "Maximum-Likelihood (False)",
    "Least Squares (True)",
    "Least Squares (False)",
]

parametrizations = [True, False, True, False, True, False, True, False]

estimators = [LinearEstimator(), 
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),]

loss_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (WeightedRelativeEntropy(3),WeightedRelativeEntropyOption("identity")),
    (WeightedRelativeEntropy(4),WeightedRelativeEntropyOption("identity")),
    (WeightedProbabilityBasedSquaredError(3), WeightedProbabilityBasedSquaredErrorOption("identity")),
    (WeightedProbabilityBasedSquaredError(4),WeightedProbabilityBasedSquaredErrorOption("identity")),
]

def generate_pgdb_algo_option():
    return ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1
    )

algo_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option())
]

In [6]:
NoiseSetting = namedtuple("NoiseSetting", ["qoperation_base", "method", "para"])
TestSetting = namedtuple(
    "TestSetting",
    [
        "tomography_class",
        "true_object",
        "tester_objects",
        "seed",
        "n_rep",
        "num_data",
        "n_sample",
        "schedules",
        "case_names",
        "estimators",
        "algo_list",
        "loss_list",
        "parametrizations"
    ],
)

noise_setting_0 = NoiseSetting(
    qoperation_base=base_state, method="depolarized", para={"error_rate": 0.1}
)
noise_setting_1 = NoiseSetting(
    qoperation_base=povm_x, method="depolarized", para={"error_rate": 0.1}
)
noise_setting_2 = NoiseSetting(
    qoperation_base=povm_y, method="depolarized", para={"error_rate": 0.1}
)
noise_setting_3 = NoiseSetting(
    qoperation_base=povm_z, method="depolarized", para={"error_rate": 0.1}
)

test_setting_0 = TestSetting(
    tomography_class=StandardQst,
    true_object=noise_setting_0,
    tester_objects=[noise_setting_1, noise_setting_2, noise_setting_3],
    seed=777,
    n_sample=2,
    n_rep=10,
    num_data=[100, 1000],
    schedules="all",
    case_names=case_names,
    estimators=estimators,
    algo_list=algo_list,
    loss_list=loss_list,
    parametrizations=parametrizations
)

# TODO: 複数のtest_settingをtest_settingsに格納する
test_settings = [test_setting_0]

In [7]:
# 1つのtest_settingから、QOperationGenerationSettingのリストを作る
GenerationSettings = namedtuple(
    "GenerationSettings", ["true_setting", "tester_settings"]
)

def to_generation_setting(noise_setting):
    # TODO: 他に良い方法がないか検討
    if noise_setting.method == "depolarized":
        target_class =  DepolarizedQOperationGenerationSetting
    else:
        message = f"noise_setting.method='{noise_setting.method}' is not implemented."
        raise NotImplementedError(message)
    return target_class(
            qoperation_base=noise_setting.qoperation_base,
            c_sys=noise_setting.qoperation_base.composite_system,
            **noise_setting.para
        )

def to_generation_settings(test_setting):
    true_setting = to_generation_setting(test_setting.true_object)
    tester_settings = [to_generation_setting(setting) for setting in test_setting.tester_objects]
    generation_settings = GenerationSettings(true_setting=true_setting, tester_settings=tester_settings)
    return generation_settings

In [8]:
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt


def generate_qtomography(sim_setting, para: bool, eps_proj_physical: float) -> "StandardQTomography":
    true_object = sim_setting.true_object
    tester_objects = sim_setting.tester_objects
    seed = sim_setting.seed

    if type(true_object) == State:
        return StandardQst(
            tester_objects,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    if type(true_object) == Povm:
        return StandardPovmt(
            tester_objects,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    if type(true_object) == Gate:
        states = [t for t in tester_objects if type(t) == State]
        povms = [t for t in tester_objects if type(t) == Povm]

        return StandardQpt(
            states=states,
            povms=povms,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    # TODO: message
    return TypeError()

In [9]:
def convert_test_setting2simulation_setting(test_setting, case_index: int) -> StandardQTomographySimulationSetting:
    return StandardQTomographySimulationSetting(
        name=test_setting.case_names[case_index],
        estimator=test_setting.estimators[case_index],
        loss=test_setting.loss_list[case_index][0],
        loss_option=test_setting.loss_list[case_index][1],
        algo=test_setting.algo_list[case_index][0],
        algo_option=test_setting.algo_list[case_index][1],
        true_object=true_object,
        tester_objects=tester_objects,
        n_rep=test_setting.n_rep,
        seed=test_setting.seed,
        num_data = test_setting.num_data,
        schedules= test_setting.schedules
    )

In [10]:
def write_result_case_unit(result: namedtuple, root_dir: str) -> None:
    test_setting_index = result.result_index["test_setting_index"]
    sample_index = result.result_index["sample_index"]
    case_index = result.result_index["case_index"]
    
    # Save all
    dir_path = Path(f"{root_dir}/{test_setting_index}/{sample_index}")
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / f"case_{case_index}_result.pickle"
    with open(path, 'wb') as f:
        pickle.dump(result, f)

    check_result = result.check_result
    path = dir_path / f"case_{case_index}_check_result.json"
    with open(path, "w") as f:
        json.dump(check_result, f, ensure_ascii=False, indent=4, separators=(',', ': '))

In [45]:
import copy

In [ ]:
def results2dataframe(results):
    pd.DataFrame()

In [84]:
def result2dict(result):
    result_dict = dict(
        test_setting_index=result.result_index["test_setting_index"],
        sample_index=result.result_index["sample_index"],
        case_index=result.result_index["case_index"],
        name=result.simulation_setting.name,
        total_result=result.check_result["total_result"])

    check_result = {}
    warning_text = ""
    for r in result.check_result["results"]:
        check_result[r["name"]] = r["result"]
    result_dict.update(check_result)
    
    return result_dict

In [82]:
pd.DataFrame([result2dict(r) for r in results])

,sample_index,case_index,name,total_result,MSE of Empirical Distributions,Consistency,MSE of estimators,Physicality Violation
0,1,0,Linear (True),True,True,True,True,True
1,1,1,Linear (False),True,True,True,True,True
2,1,2,ProjectedLinear (True),False,True,True,False,True
3,1,3,ProjectedLinear (False),False,True,True,False,True
4,1,4,Maximum-Likelihood (True),False,True,True,False,True
5,1,5,Maximum-Likelihood (False),False,True,True,False,True
6,1,6,Least Squares (True),True,True,True,True,True
7,1,7,Least Squares (False),True,True,True,True,True


In [85]:
# QOperationGenerationSettingから、qoperationのサンプルをn_sample個作る
Result = namedtuple(
    "Result",
    ["result_index", "simulation_setting", "estimation_results", "check_result"],
)

test_setting_i = 0
generation_settings = to_generation_settings(test_settings[test_setting_i])
seed = test_settings[test_setting_i].seed
n_sample = test_settings[test_setting_i].n_sample
test_setting = test_settings[test_setting_i]

for sample_i in range(n_sample):
    print(f"[{sample_i+1}/{n_sample}] Generating sample ...")

    # sampleの生成
    true_object = generation_settings.true_setting.generate()
    tester_objects = [
        tester_setting.generate()
        for tester_setting in generation_settings.tester_settings
    ]

    results = []
    case_n = len(test_setting.case_names)
    for case_index in range(case_n):
        # QTomographySimulationSettingを作る
        sim_setting = convert_test_setting2simulation_setting(test_setting, case_index)
        print(f"Case {case_index}: {sim_setting.name}")

        org_sim_setting = copy.deepcopy(sim_setting)
        
        # QTomographyの作成
        qtomography = generate_qtomography(
            sim_setting,
            para=test_setting.parametrizations[case_index],
            eps_proj_physical=1e-13,
        )

        # 推定
        estimation_results = data_analysis.execute_simulation(
            qtomography=qtomography, simulation_setting=sim_setting
        )

        # 自動テスト
        sim_check = StandardQTomographySimulationCheck(sim_setting, estimation_results)
        check_result = sim_check.execute_all(show_detail=False, with_detail=True)

        # 結果の表示
        if not check_result["total_result"]:
            start_red = "\033[31m"
            end_color = "\033[0m"
            print(f"Total Result: {start_red}NG{end_color}")

        result_index = dict(
            test_setting_index=test_setting_i,
            sample_index=sample_i,
            case_index=case_index,
        )
        # 結果の格納
        result = Result(
            result_index=result_index,
            simulation_setting=org_sim_setting,
            estimation_results=estimation_results,
            check_result=check_result,
        )
        results.append(result)

        # 保存
        write_result_case_unit(result, root_dir="result")

    # 保存（sample_index単位）
    # 保存先: {root_dir}/{test_index}/{sample_index}/result.csv
    # path = test_setting_i}
    
    # Save
    result_dict_list = [result2dict(r) for r in results]
    sample_result_df = pd.DataFrame(result_dict_list)
    sample_result_df.to_csv("check_results.csv")
    
    

# 保存（test_index単位）
# 保存先: {root_dir}/{test_index}/result.csv

[1/2] Generating sample ...
Case 0: Linear (True)


  0%|          | 0/10 [00:00<?, ?it/s]

========== Summary ============
Name: Linear (True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 1: Linear (False)


 30%|███       | 3/10 [00:00<00:00, 29.97it/s]

========== Summary ============
Name: Linear (False)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 2: ProjectedLinear (True)


...

========== Summary ============
Name: Maximum-Likelihood (False)
MSE of Empirical Distributions: OK
Consistency: NG (to_be_checked=False)
MSE of estimators: NG
Physicality Violation: OK
Total Result: NG
Case 6: Least Squares (True)


  0%|          | 0/10 [00:00<?, ?it/s]

========== Summary ============
Name: Least Squares (True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 7: Least Squares (False)


100%|██████████| 10/10 [00:00<00:00, 23523.86it/s]

========== Summary ============
Name: Least Squares (False)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK


In [68]:
import pandas as pd

In [69]:
data = [dict(a=1, b=1), dict(a=2, b=2)]

pd.DataFrame(data)

,a,b
0,1,1
1,2,2


In [6]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_povm(c_sys)
povm_y = get_y_povm(c_sys)
povm_z = get_z_povm(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [7]:
# Case 1:
# true_object = get_z0_1q(c_sys)

# Case 2:
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)

# Case 3:
# vec = np.array([1 / np.sqrt(2), 0, 0, 0], dtype=np.float64)
# true_object = State(c_sys, vec)

true_object.vec

array([0.70710678, 0.40824829, 0.40824829, 0.40824829])

In [8]:
num_data = [100, 1000]
n_rep = 1000

case_name_list = [
    "Linear(True)",
    "Linear(False)",
    "ProjectedLinear(True)",
    "ProjectedLinear(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=True,
        eps_proj_physical=Settings.get_atol(),
        seed=seed,
    ),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=False,
        eps_proj_physical=Settings.get_atol(),
        seed=seed,
    ),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=True,
        eps_proj_physical=Settings.get_atol(),
        seed=seed,
    ),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=False,
        eps_proj_physical=Settings.get_atol(),
        seed=seed,
    ),
]
para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(3),
    WeightedRelativeEntropy(4),
    WeightedProbabilityBasedSquaredError(3),
    WeightedProbabilityBasedSquaredError(4),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption("identity"),
    WeightedRelativeEntropyOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
    ProjectedGradientDescentBacktrackingOption(),
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.execute_simulation(qtomography=qtomography, simulation_setting=simulation_setting)
    estimation_results_list.append(estimation_results)

    # stock settings of this simulation
    simulation_setting = StandardQTomographySimulationSetting(
        name=name,
        true_object=true_object,
        tester_objects=tester_objects,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
    )
    simulation_settings.append(simulation_setting)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  0%|          | 0/1000 [00:00<?, ?it/s]

Case 0: Linear(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


  1%|          | 9/1000 [00:00<00:11, 85.75it/s]

elapsed_time:0.20819623470306398[min]

Case 1: Linear(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


  0%|          | 4/1000 [00:00<00:27, 36.55it/s]

...

elapsed_time:8.888639851411183[min]

Case 6: Least Squares(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LossMinimizationEstimator


  0%|          | 1/1000 [00:00<02:10,  7.64it/s]

elapsed_time:1.6980049331982932[min]

Case 7: Least Squares(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LossMinimizationEstimator


100%|██████████| 1000/1000 [02:29<00:00,  6.71it/s]


elapsed_time:2.4838359196980795[min]



In [9]:
case_name = "case_2_軸から離れた表面"
report.export_report(f"qst_sample_{case_name}_nrep={n_rep}.pdf",
    estimation_results_list=estimation_results_list,   # 「EstimationResultのリスト」のリスト
    simulation_settings= simulation_settings,
    # true_object=true_object,  # True Object
    # tester_objects = tester_objects,   # Tester Objectのリスト.
    seed=seed,  # 推定で使ったseed（オプション）
    tolerance=Settings.get_atol()
)

​Generating table of computation time ...
​Generating table of tolerance of physicality constraint violation ...
​Generating table of experimental conditions ...
Generating case list ...
Computation time of estimators ...


100%|██████████| 1000/1000 [00:00<00:00, 175405.82it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  Linear(True)=[0.009929648467459593, 0.0010292166444478218]
  Linear(False)=[0.009930903279026853, 0.0010002055818730709]
  ProjectedLinear(True)=[0.007628986960280322, 0.0008336141767192117]
  ProjectedLinear(False)=[0.00773715441575743, 0.000816657724263995]
  Maximum-Likelihood(True)=[0.008499421341326589, 0.0008414304279898206]
  Maximum-Likelihood(False)=[0.008167519972991207, 0.0008395604755059923]
  Least Squares(True)=[0.007995055665783955, 0.0007773434372293063]
  Least Squares(False)=[0.007853440311890569, 0.0008198814867697979]
  Linear(True)=[0.009929648467459593, 0.0010292166444478218]
  ProjectedLinear(True)=[0.007628986960280322, 0.0008336141767192117]
  Maximum-Likelihood(True)=[0.008499421341326589, 0.0008414304279898206]
  Least Squares(True)=[0.007995055665783955, 0.0007773434372293063]
  Linear(False)=[0.009930903279026853, 0.0010002055818730709]
  ProjectedLinear(False)=[0.00773715441575743, 

100%|██████████| 1000/1000 [00:00<00:00, 8011.74it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (qst_sample_case_2_軸から離れた表面_nrep=1000.pdf)


In [10]:
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

In [12]:
i = 7
sim_check = StandardQTomographySimulationCheck(simulation_settings[i], estimation_results_list[i])
sim_check. execute_all()

100%|██████████| 1000/1000 [00:00<00:00, 183718.97it/s]
/Users/tomoko/project/rcast/workspace/quara/quara/data_analysis/simulation_check.py:122: UserWarning:

Estimator MSE is not checked except for LinearEstimator, ProjectedLinearEstimator, Maximum-likelihood.



[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 4.203786964071532e-06
eps = 0.024565513539877085
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 3.503762214336105e-06
eps = 0.0024231444828227097
[OK] Consistency Check
result=1.9036745341808358e-16
eps=1e-12
result < eps: True
[OK] N=100 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13
[OK] N=100 is_ineq_constraint_satisfied_all
True=1000, False=0, eps=1e-09
[OK] N=1000 is_ineq_constraint_satisfied_all
True=1000, False=0, eps=1e-09
========== Summary ============
Name: Least Squares(False)
MSE of Empirical Distributions: OK
Consistency: OK
MSE of estimators: OK
Physicality Violation: OK


True

In [16]:
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

results = []
for i in range(len(simulation_settings)):
    sim_check = StandardQTomographySimulationCheck(simulation_settings[i], estimation_results_list[i])
    print(f"Case {i}: {simulation_settings[i].name}")
    result = sim_check.execute_all()
    results.append(result)
    print("==============================")
    print()

100%|██████████| 1000/1000 [00:00<00:00, 179044.82it/s]

Case 0: Linear(True)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 7.035153254041429e-05
eps = 0.024438329256302023
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 2.9216644447820506e-05
eps = 0.002518294321219499
[OK] Consistency Check
result=9.244463733058732e-33
eps=1e-16
result < eps: True


[OK] N=100
|MSE_Linear - MSE_Analytical| = 7.035153254040562e-05
eps = 0.024438329256302037
[OK] N=1000
|MSE_Linear - MSE_Analytical| = 2.9216644447821807e-05
eps = 0.0025182943212194997
[OK] N=100 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13


100%|██████████| 1000/1000 [00:00<00:00, 145585.01it/s]

[OK] N=1000 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13
========== Summary ============
Name: Linear(True)
MSE of Empirical Distributions: OK
Consistency: OK
MSE of estimators: OK
Physicality Violation: OK

Case 1: Linear(False)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 6.90967209731521e-05
eps = 0.024780023013208464

...


/Users/tomoko/project/rcast/workspace/quara/quara/data_analysis/simulation_check.py:122: UserWarning:

Estimator MSE is not checked except for LinearEstimator, ProjectedLinearEstimator, Maximum-likelihood.



[OK] N=100 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13
[NG] N=100 is_ineq_constraint_satisfied_all
True=439, False=561, eps=1e-09


100%|██████████| 1000/1000 [00:00<00:00, 197816.54it/s]

[NG] N=1000 is_ineq_constraint_satisfied_all
True=472, False=528, eps=1e-09
========== Summary ============
Name: Least Squares(True)
MSE of Empirical Distributions: OK
Consistency: OK
MSE of estimators: OK
Physicality Violation: NG

Case 7: Least Squares(False)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 4.203786964071532e-06
eps = 0.024565513539877085
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 3.503762214336105e-06
eps = 0.0024231444828227097
[OK] Consistency Check
result=1.9036745341808358e-16
eps=1e-12
result < eps: True


[OK] N=100 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=1000, False=0, eps=1e-13
[OK] N=100 is_ineq_constraint_satisfied_all
True=1000, False=0, eps=1e-09
[OK] N=1000 is_ineq_constraint_satisfied_all
True=1000, False=0, eps=1e-09
========== Summary ============
Name: Least Squares(False)
MSE of Empirical Distributions: OK
Consistency: OK
MSE of estimators: OK
Physicality Violation: OK



In [22]:
from quara.objects.gate import get_depolarizing_channel

In [54]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

actual = get_depolarizing_channel(0.1, c_sys)
actual.hs

id(c_sys)=4810395856
id(gate.composite_system)=4810395856


array([[1. , 0. , 0. , 0. ],
       [0. , 0.9, 0. , 0. ],
       [0. , 0. , 0.9, 0. ],
       [0. , 0. , 0. , 0.9]])

In [57]:
actual.composite_system is c_sys

True

In [55]:
id(c_sys)

4810395856

In [56]:
id(actual.composite_system)

4810395856

In [51]:
id(a.composite_system.elemental_systems[0])

4682089616

In [52]:
id(c_sys.elemental_systems[0])

4811143776

In [58]:
e_sys_1 = ElementalSystem(0, get_normalized_pauli_basis())
e_sys_2 = ElementalSystem(1, get_normalized_pauli_basis())
c_sys_2q = CompositeSystem([e_sys_1, e_sys_2])

In [59]:
actual = get_depolarizing_channel(0.1, c_sys_2q)

In [64]:
c_sys_2q.dim

4

In [67]:
expected = np.diag([0.9] * 16)
expected[0][0] = 1
expected

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.

In [63]:
actual.hs

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.

In [65]:
np.array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0.9, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0.9, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0.9]])

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.

In [69]:
from quara.data_analysis.generation_setting import DepolarizedQOperationGenerationSetting

In [71]:
true_object

In [81]:
setting = DepolarizedQOperationGenerationSetting(c_sys, true_object, 0.1)

In [83]:
setting.generate()

In [84]:
source_povm = tester_objects[0]

In [94]:
setting = DepolarizedQOperationGenerationSetting(c_sys, source_povm, 0.1)
setting.generate()

type(elem1)=<class 'quara.objects.povm.Povm'>, type(elem2)=<class 'quara.objects.gate.Gate'>


In [89]:
from quara.objects.gate import get_x
source_gate = get_x(c_sys)
source_gate.hs

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  0.],
       [ 0.,  0.,  0., -1.]])

In [91]:
setting = DepolarizedQOperationGenerationSetting(c_sys, source_gate, 0.1)
setting.generate().hs

type(elem1)=<class 'quara.objects.gate.Gate'>, type(elem2)=<class 'quara.objects.gate.Gate'>


array([[ 1. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.9,  0. ,  0. ],
       [ 0. ,  0. , -0.9,  0. ],
       [ 0. ,  0. ,  0. , -0.9]])